Install the package via "pip install opencosmorspy"

Import the module

In [1]:
import numpy as np

from opencosmorspy import Parameterization, COSMORS

Choose a default parameterization (default_turbomole or default_orca). You can either instanciate a parameterization object and pass it to the COSMORS constructor, or let the COSMORS class take over the instanciation.

In [2]:
crs = COSMORS(par='default_orca')

To modify parameters, currently indivual properties of the parameters class are overwritten. 

In [3]:
crs.par.calculate_contact_statistics_molecule_properties = True

In [4]:
print(crs.par)

qc_program             : orca
descriptors            : sigma, sigma_orth, elmnt_nr, group, mol_charge 
a_eff                  : 6.226
r_av                   : 0.500
sigma_step             : 0.001
sigma_orth_step        : 0.001
mf_use_sigma_orth      : True
mf_alpha               :  7.579075e+06
mf_r_av_corr           : 1.000
mf_f_corr              : 2.400
hb_term                : default
hb_c                   :  2.748875e+07
hb_c_T                 : 1.500
hb_sigma_thresh        : 0.007686
hb_don_elmnt_nr_arr    : [  1 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116
 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134
 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150]
hb_acc_elmnt_nr_arr    : [ 6  7  8  9 15 16 17 35 53]
hb_pref_arr            : [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 

Add a molecule:

In [5]:
mol_structure_list_0 = ['./../tests/COSMO_ORCA/C2H2Cl4_001_1112tetrachloroethane/COSMO_TZVPD/C2H2Cl4_001_1112tetrachloroethane_CnfS1_c000.orcacosmo']
crs.add_molecule(mol_structure_list_0)

You can clear the list of molecules at any stage:

In [6]:
crs.clear_molecules()

But now we do want to add two molecules. At present, only one molecular structure per molecule is supported.

In [7]:
mol_structure_list_0 = ['./../tests/COSMO_ORCA/C2H2Cl4_001_1112tetrachloroethane/COSMO_TZVPD/C2H2Cl4_001_1112tetrachloroethane_CnfS1_c000.orcacosmo']
crs.add_molecule(mol_structure_list_0)
crs.add_molecule(['./../tests/COSMO_ORCA/H2O_001/COSMO_TZVPD/H2O_c000.orcacosmo'])

Let us add two jobs:

In [8]:
x = np.array([0.0, 1.0])
T = 298.15
crs.add_job(x, T, refst='pure_component')

In [9]:
x = np.array([0.5, .5])
T = 298.15
crs.add_job(x, T, refst='pure_component')

In [10]:
results = crs.calculate()

In [11]:
print('Total logarithmic activity coefficient: ', results['tot']['lng'])
print('Residual logarithmic activity coefficient: ', results['enth']['lng'])
print('Combinatorial logarithmic activity coefficient:', results['comb']['lng'])

Total logarithmic activity coefficient:  [[9.24958484 0.        ]
 [1.03917659 1.58421651]]
Residual logarithmic activity coefficient:  [[10.02243494  0.        ]
 [ 1.14511374  1.78316791]]
Combinatorial logarithmic activity coefficient: [[-0.7728501   0.        ]
 [-0.10593715 -0.1989514 ]]


To view partial molar properties and average interaction energies:

In [12]:
print('Partial molar           - pm_A_int: \n', results['enth']['pm_A_int'], '\n')
print('Partial molar           - pm_E_int: \n', results['enth']['pm_E_int'], '\n')
print('Partial molar           - pm_E_hb: \n', results['enth']['pm_E_hb'], '\n')
print('Partial molar           - pm_E_mf: \n', results['enth']['pm_E_mf'], '\n')
print('Average interaction mol - aim_A_int: \n', results['enth']['aim_A_int'], '\n')

Partial molar           - pm_A_int: 
 [[7640.62763607    0.        ]
 [1308.08119381 1237.37604317]] 

Partial molar           - pm_E_int: 
 [[11771.91298266     0.        ]
 [ 2361.76930712  1962.96861731]] 

Partial molar           - pm_E_hb: 
 [[6885.47557766    0.        ]
 [1756.14685552 1209.3209569 ]] 

Partial molar           - pm_E_mf: 
 [[4886.437405      0.        ]
 [ 605.6224516   753.64766041]] 

Average interaction mol - aim_A_int: 
 [[3091.9226888     0.        ]
 [ 137.65463075 2407.80101377]] 



You may also use different reference states for the molecules. To use a fixed concentration referene mixture:

In [13]:
crs.clear_jobs()

x = np.array([0.0, 1.0])
T = 298.15
crs.add_job(x, T, refst='reference_mixture', x_refst=np.array([0, 1.]))

x = np.array([0.5, 0.5])
T = 298.15
crs.add_job(x, T, refst='reference_mixture', x_refst=np.array([0, 1.]))

results = crs.calculate()

print('Total activity coefficient: \n', results['tot']['lng'], '\n')

Total activity coefficient: 
 [[ 0.          0.        ]
 [-8.21040825  1.58421651]] 



To use the COSMO reference state directly:

In [14]:
crs.clear_jobs()
x = np.array([0.5, 0.5])
T = 298.15
crs.add_job(x, T, refst='cosmo', x_refst=np.array([0, 1.]))
results = crs.calculate()

crs.clear_jobs()
x = np.array([0.0, 1.0])
T = 298.15
crs.add_job(x, T, refst='cosmo', x_refst=np.array([0, 1.]))
results_ref = crs.calculate()

In [15]:
print('Total activity coefficient: \n', results['tot']['lng']-results_ref['tot']['lng'], '\n')

Total activity coefficient: 
 [[-8.21040825  1.58421651]] 

